<a href="https://colab.research.google.com/github/ollihansen90/linclassifiers/blob/main/Futureskills/LinClass_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kapitel 7 - Lineare Klassifikation

## Setup

In [ ]:
# Todo: utils-Datei anlegen, die am Anfang geladen wird
# utils_linclass.py
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from IPython.display import display
from PIL import Image
from IPython.display import Image as Image_

class Koordinatensystem():
    def __init__(self, config={}):
        self.config = {
            "size": 11,
            "grid_alpha": 0.1,
            }
        self.config.update(config)

    def draw(self):
        size = self.config["size"]
        grid_alpha = self.config["grid_alpha"]
        plt.plot([0,0], [-size,size], "k")
        plt.plot([-size,size], [0,0], "k")
        for t in range(1,size):
            plt.plot([t,t], [-size,size], "k", alpha=grid_alpha)
            plt.plot([-t,-t], [-size,size], "k", alpha=grid_alpha)
            plt.plot([-size,size], [t,t], "k", alpha=grid_alpha)
            plt.plot([-size,size], [-t,-t], "k", alpha=grid_alpha)
            if t%2==0:
                plt.plot([t,t], [-0.1,0.1], "k")
                plt.text(t,-0.5, str(t), {"horizontalalignment":"center", "verticalalignment":"center"})
                plt.plot([-t,-t], [-0.1,0.1], "k")
                plt.text(-t,-0.5, str(-t), {"horizontalalignment":"center", "verticalalignment":"center"})
                plt.plot([-0.1,0.1], [t,t], "k")
                plt.text(-0.4, -t, str(-t), {"horizontalalignment":"right", "verticalalignment":"center"})
                plt.plot([-0.1,0.1], [-t,-t], "k")
                plt.text(-0.4, t, str(t), {"horizontalalignment":"right", "verticalalignment":"center"})
        plt.plot([-0.1,0,0.1], [size-0.2,size,(size-0.2)], "k")
        plt.plot([(size-0.2),size,(size-0.2)], [-0.1,0,0.1], "k")
        plt.plot([-0.1,0,0.1], [-(size-0.2),-size,-(size-0.2)], "k")
        plt.plot([-(size-0.2),-size,-(size-0.2)], [-0.1,0,0.1], "k")
        plt.text(-0.4, (size-0.2), "y", {"horizontalalignment":"center", "verticalalignment":"center"})
        plt.text(size, -0.4, "x", {"horizontalalignment":"center", "verticalalignment":"center"})

class Vektorfolge():
    def __init__(self, veclist, colorlist=list(mcolors.TABLEAU_COLORS.keys()), alphalist=None):
        self.veclist = veclist
        self.colorlist = colorlist
        self.alphalist = [1]*len(veclist) if alphalist==None else alphalist
        self.config = {"angles":'xy', "scale_units":'xy', "scale":1, "width":0.005, "zorder":2}

    def draw(self):
        plt.quiver(0,0,*self.veclist[0],**self.config, color=self.colorlist[0], alpha=self.alphalist[0])
        v = self.veclist[0].copy()
        for i in range(1, len(self.veclist)):
            plt.quiver(
                *v,
                *self.veclist[(i)],
                **self.config,
                color=self.colorlist[(i)%len(self.colorlist)],
                alpha=self.alphalist[i]
                )
            v += self.veclist[i].copy()

class Gerade():
    def __init__(self, aufpunkt, richtung):
        assert aufpunkt.shape==np.zeros(2).shape, "Der Vektor für den Aufpunkt hat nicht die richtige Form."
        assert richtung.shape==np.zeros(2).shape, "Der Vektor für die Richtung hat nicht die richtige Form."
        assert np.sum(np.abs(richtung))>0, "Der Vektor darf nicht [0,0] sein."
        self.aufpunkt = aufpunkt
        self.richtung = richtung

    def draw(self, col="tab:blue"):
        size = 11
        ecke1 = self.aufpunkt.copy()
        while np.max(np.abs(ecke1))<=size:
            ecke1 += self.richtung
        ecke2 = self.aufpunkt.copy()
        while np.max(np.abs(ecke2))<=size:
            ecke2 -= self.richtung
        ecken = np.stack([ecke1, ecke2])
        plt.plot(ecken[:,0], ecken[:,1], col)

    def on_gerade(self, pt):
        p = pt.copy().astype(float)
        p -= self.aufpunkt
        if np.sum(p**2)==0:
            return True
        p /= np.linalg.norm(p)
        return np.abs((self.richtung/np.linalg.norm(self.richtung))@p)>1-1e-6

def generate_data():
    np.random.seed(1)

    N = 100
    data = 3*np.random.randn(N,2)
    data[:N//2] += np.array([-4,6])
    data[N//2:] += np.array([2,-8])
    data = np.column_stack([data, -np.ones(N)])
    label = np.array(N//2*[1]+N//2*[-1])

    # Permutiere Daten
    perm = np.random.permutation(len(label))
    data = data[perm]
    label = label[perm]

    return data, label

def show_vid(vid):
    every = max((len(vid)//50, 1))
    vid = np.stack(vid[::every])
    vid = [Image.fromarray(img) for img in vid]
    vid[0].save("array.gif", save_all=True, append_images=vid[1:], duration=50, loop=0)

    with open('/content/array.gif','rb') as f:
        display(Image_(data=f.read(), format='gif'))

In [ ]:
# Todo: utils-Datei anlegen, die am Anfang geladen wird
# utils_linclass_07.py
def draw1(gewichtsvektor=np.ones(2), theta=0, first=False):
    data = np.array([[3,6],[-2,7]])

    size = 11
    config = {"size": size}
    koordinatensystem = Koordinatensystem(config)
    #aufpunkt = # theta = normal@aufpunkt = n1a1+n2a2
    if gewichtsvektor[1]==0:
        aufpunkt = np.array([(theta-gewichtsvektor[1])/gewichtsvektor[0],1])
    else:
        aufpunkt = np.array([1,(theta-gewichtsvektor[0])/gewichtsvektor[1]])
    gerade = Gerade(aufpunkt, gewichtsvektor[::-1]*np.array([1,-1]))
    vec = Vektorfolge([aufpunkt, gewichtsvektor], ["blue", "tab:green"], alphalist=[0,1])

    plt.figure(figsize=[7,7])
    koordinatensystem.draw()
    if not first:
        gerade.draw()
        vec.draw()
    plt.plot(data[:,0], data[:,1], "ro", markersize=5)
    plt.axis("equal");plt.xlim([-size,size]);plt.ylim([-size,size]);plt.axis("off")
    plt.show()
    #print(data)
    if not first:
        if gerade.on_gerade(data[0]) and gerade.on_gerade(data[1]):
            print(f"Sehr gut! Gewichtsvektor [{gewichtsvektor[0]},{gewichtsvektor[1]}] und  Schwellenwert {theta} sind korrekt!")
            return True
        else:
            print(f"Das stimmt leider noch nicht... Gewichtsvektor [{gewichtsvektor[0]},{gewichtsvektor[1]}] und Schwellenwert {theta} sind nicht richtig.")
            return False

def draw2(gewichtsvektor=np.ones(2), theta=0, first=False):
    data, label = generate_data()

    size = 11
    config = {"size": size}
    koordinatensystem = Koordinatensystem(config)
    #aufpunkt = # theta = normal@aufpunkt = n1a1+n2a2
    if gewichtsvektor[1]==0:
        aufpunkt = np.array([(theta-gewichtsvektor[1])/gewichtsvektor[0],1])
    else:
        aufpunkt = np.array([1,(theta-gewichtsvektor[0])/gewichtsvektor[1]])
    gerade = Gerade(aufpunkt, gewichtsvektor[::-1]*np.array([1,-1]))
    vec = Vektorfolge([aufpunkt, gewichtsvektor], ["blue", "tab:green"], alphalist=[0,1])

    plt.figure(figsize=[7,7])
    koordinatensystem.draw()
    if not first:
        gerade.draw(col="r")
        vec.draw()
    plt.scatter(data[label==1,0], data[label==1,1])
    plt.scatter(data[label==0,0], data[label==0,1])
    plt.axis("equal");plt.xlim([-size,size]);plt.ylim([-size,size]);plt.axis("off")
    plt.show()
    #print(data)
    if not first:
        out = 1*((data@gewichtsvektor-theta)>=0)
        acc = np.sum(out==label)/N
        if acc==1:
            print(f"Sehr gut! Gewichtsvektor [{gewichtsvektor[0]},{gewichtsvektor[1]}] und  Schwellenwert {theta} sind korrekt!")
            return True
        else:
            print(f"Das stimmt leider noch nicht... Gewichtsvektor [{gewichtsvektor[0]},{gewichtsvektor[1]}] und Schwellenwert {theta} sind nicht richtig. \nDer Klassifikator hätte eine Genauigkeit von {acc*100}%, {int(N*(1-acc))} Punkte werden also falsch klassifiziert.")
            return False

In [ ]:
from IPython.display import clear_output
import numpy as np
# from utils import
#from utils_linclass_07 import draw1, draw2

In diesem Notebook soll das erworbene Wissen aus Kapitel 7 angewendet werden.

## Aufgabe 1 - Gerade durch zwei Punkte
Gegeben sind zwei Punkte, durch die eine Gerade verlaufen soll. Die Gerade hat die Form $g(\vec{x})=\vec{w}\odot\vec{x}-\theta$ mit Gewichtsvektor $\vec{w}$ und Schwellenwert $\theta$.

Starten Sie den folgenden Codeblock und geben Sie einen Gewichtsvektor, sowie einen Schwellenwert ein. Ein Vektor wird hierbei komponentenweise mit Komma getrennt eingegeben. Der Vektor $\vec{x}=[1,-1]$ wäre beispielsweise `1,-1`. Beim Threshold handelt es sich nur um eine einzelne Zahl. Zahlen mit Nachkommastellen müssen mit Punkt geschrieben werden (statt beispielsweise `-2,3` ist die Eingabe `-2.3`).

In [ ]:
draw1(first=True)

done = False
while not done:
    print("Geben Sie einen Gewichtsvektor ein. (Zum Beispiel 1,-1)")
    test = input("Gewichtsvektor: ")
    gewichtsvektor = np.array([float(s) for s in test.split(",")])
    print("Geben Sie einen Schwellenwert ein. (Zum Beispiel -0.5)")
    test = input("Schwellenwert: ")
    theta = float(test)

    clear_output()
    done = draw1(gewichtsvektor, theta)


## Aufgabe 2: Punktewolken
Genau wie in Aufgabe 1 soll hier eine Gerade in der Form $g(\vec{x})=\vec{w}\odot\vec{x}-\theta$ mit Gewichtsvektor $\vec{w}$ und Schwellenwert $\theta$ aufgestellt werden. Gegeben werden zwei Punktewolken, die linear getrennt werden sollen. Die blaue Punktewolke soll die Klassenzugehörigkeit `+1` bekommen, die orangene Punktewolke erhält Klassenzugehörigkeit `-1`.

Starten Sie den folgenden Codeblock und geben Sie einen Gewichtsvektor, sowie einen Schwellenwert ein. Ein Vektor wird hierbei komponentenweise mit Komma getrennt eingegeben. Der Vektor $\vec{x}=[1,-1]$ wäre beispielsweise `1,-1`. Beim Threshold handelt es sich nur um eine einzelne Zahl. Zahlen mit Nachkommastellen müssen mit Punkt geschrieben werden (statt beispielsweise `-2,3` ist die Eingabe `-2.3`).

In [ ]:
draw2(first=True)

done = False
while not done:
    print("Geben Sie einen Gewichtsvektor ein. (Zum Beispiel 1,-1)")
    test = input("Gewichtsvektor: ")
    gewichtsvektor = np.array([float(s) for s in test.split(",")])
    print("Geben Sie einen Schwellenwert ein. (Zum Beispiel -0.5)")
    test = input("Schwellenwert: ")
    theta = float(test)

    clear_output()
    done = draw2(gewichtsvektor, theta)
